In [5]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple/
  Using cached tensorboard-2.11.2-py3-none-any.whl (6.0 MB)
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.10.1
    Uninstalling tensorboard-2.10.1:
      Successfully uninstalled tensorboard-2.10.1


In [6]:
!pip install tensorboard

Looking in indexes: https://pypi.org/simple/


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.datasets import fashion_mnist, cifar10
from keras.layers import Dense, Flatten, Normalization, Dropout, Conv2D, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom, BatchNormalization, Activation, InputLayer
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import utils
import os
from keras.preprocessing.image import ImageDataGenerator

import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import tensorboard

ImportError: cannot import name 'experimental_functions_run_eagerly' from 'tensorflow.python.eager.def_function' (c:\Users\Elsa\anaconda3\lib\site-packages\tensorflow\python\eager\def_function.py)

# Tensorboard and Pretrained Models

As we move into models that are more complex, training processes that take longer, and predictive challenges that are more difficult, we can add some new tools that can make our lives easier. In this notebook, we'll look at how to use Tensorboard to visualize our training process, and how to use pretrained models to improve our results.

In [9]:
# Load Some Data
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)

NameError: name 'tf' is not defined

## Tensorboard

Tensorboard is a tool from Keras that can monitor the results of a tensorflow model and display it in a nice Tableau-like dashboard view. We can enable tensorboard and add it to our modelling process to get a better view of progress and save on some of the custom charting functions. 

![Tensorboard](images/tensorboard.gif "Tensorboard")

The first thing that we can use tensorboard for is to get a nice chart of our training progress. 

### Create Model

We'll create a model, and spin up a tensorboard instance to monitor it.

In [9]:
# Set # of epochs
epochs = 10

In [10]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [11]:
acc = keras.metrics.CategoricalAccuracy(name="accuracy")
pre = keras.metrics.Precision(name="precision")
rec = keras.metrics.Recall(name="recall")
metric_list = [acc, pre, rec]

#### Add Tensorboard Callback

The tensorboard can be added to the model as it is being fit as a callback. The primary parameter that matters there is the log_dir, where we can setup the folder to put the logs that the visualizations are made from. The example I have here is from the tensorflow documentation, generating a new subfolder for each execution. Using this to log the tensorboard data is fine, there's no need to change it without reason. 

### Launch Tensorboard

In recent versions of VS Code, whioch I assume all of you have, tensorboard can be used directly in a VS Code tab:

![VS Code Tensor](images/vscode_tensorboard.png "VS Code Tensor" )

The command below launches tensorboard elsewhere, such as Google colab.

Either way, the actual tensorboard feature works the same once launched. We can open it before or after we start training the model. If we open it before we can update it to watch training progress - something that may be usefull if you have models that can train for a very long time. 

In [12]:
%load_ext tensorboard
%tensorboard --logdir logs/fit
# The logdir is wherever the logs are, this is specified in the callback setup. 

Reusing TensorBoard on port 6006 (pid 60367), started 0:25:15 ago. (Use '!kill 60367' to kill it.)

In [14]:
model = create_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=metric_list)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=epochs, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/10
1875/1875 [==============================] - 8s 3ms/step - loss: 0.2189 - accuracy: 0.9420 - precision: 0.9592 - recall: 0.9254 - val_loss: 0.1121 - val_accuracy: 0.9655 - val_precision: 0.9725 - val_recall: 0.9609
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0973 - accuracy: 0.9703 - precision: 0.9752 - recall: 0.9656 - val_loss: 0.0852 - val_accuracy: 0.9732 - val_precision: 0.9767 - val_recall: 0.9701
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0696 - accuracy: 0.9774 - precision: 0.9803 - recall: 0.9752 - val_loss: 0.0737 - val_accuracy: 0.9773 - val_precision: 0.9794 - val_recall: 0.9744
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0526 - accuracy: 0.9835 - precision: 0.9852 - recall: 0.9817 - val_loss: 0.0695 - val_accuracy: 0.9792 - val_precision: 0.9805 - val_recall: 0.9778
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0436 - accuracy: 0

### Tensorboard Contents

The first page of the tensorboard page gives us a nice pretty view of our training progress - this part should be quite straightforward. The board will capture whatever executions are in that log file, we can filter them on the side to see what we are currently working on, or use different log locations to keep things separate. 

Like the text results, we get whichever metrics were specified when setting up the model. 

#### Tensorboard Images

We can also use the tensorboard to visualize other stuff. For example we can load up some images from our dataset. 

In [15]:
# Sets up a timestamped log directory.

logdir = "logs/train_data/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

In [16]:
with file_writer.as_default():
    # Don't forget to reshape.
    images = np.reshape(x_train[0:25], (-1, 28, 28, 1))
    tf.summary.image("25 training data examples", images, max_outputs=25, step=0)

## Using Pretrained Models

As we've seen lately, training neural networks can take a really long time. Highly accurate models such as the ones that are used for image recognition in a self driving cars can take multiple computers days or weeks to train. With one laptop we don't really have the ability to get anywhere close to that. Is there any hope of getting anywhere near that accurate?

We can use models that have been trained on large datasets and adapt them to our purposes. By doing this we can benefit from all of that other learning that is embedded into a model without going through a training process that would be impossible with our limited resources. 

We will look at using a pretrained model here, and at making modifications to it next time. 

#### Functional Models

I have lied to you, I forgot that the pretrained models are not sequntial ones (generally, not as a rule), so some of the syntax here is for functional models. It leads to us using some slightly unfamiliar syntax. 

In [10]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)


Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


### Download Model

There are several models that are pretrained and available to us to use. VGG16 is one developed to do image recognition, the name stands for "Visual Geometry Group" - a group of researchers at the University of Oxford who developed it, and ‘16’ implies that this architecture has 16 layers. The model got ~93% on the ImageNet test that we mentioned a couple of weeks ago. 

![VGG16](images/vgg16.png "VGG16" )

In [11]:
# Load Model
from keras.applications.vgg16 import VGG16
from keras.layers import Input
from keras.models import Model

input_tensor = Input(shape=(160, 160, 3))
vgg = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense(1, activation='sigmoid')(x)

model = Model(inputs=vgg.input, outputs=prediction)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 160, 160, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 160, 160, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 80, 80, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 80, 80, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 80, 80, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 40, 40, 128)       0     

In [12]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=metric_list)

log_dir = "logs/fit/VGG" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_dataset, 
          epochs=epochs, 
          validation_data=validation_dataset, 
          callbacks=[tensorboard_callback])
          
model.evaluate(validation_dataset)

Epoch 1/10
63/63 [==============================] - 300s 5s/step - loss: 0.0000e+00 - accuracy: 0.9853 - precision: 0.9816 - recall: 0.8938 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
63/63 [==============================] - 251s 4s/step - loss: 0.0000e+00 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
63/63 [==============================] - 439s 7s/step - loss: 0.0000e+00 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/10
40/63 [==================>...........] - ETA: 1:34 - loss: 0.0000e+00 - accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00

In [ ]:
model.evaluate(validation_dataset)

## More Complex Data

We can use the rose data for a more complex dataset and a more interesting example in terms of accuracy. 

In [ ]:
import pathlib
import PIL 

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)

#Flowers
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

In [ ]:
input_tensor = Input(shape=(180, 180, 3))
vgg = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense(5)(x)

model = Model(inputs=vgg.input, outputs=prediction)

model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
            optimizer="adam", 
            metrics=keras.metrics.SparseCategoricalAccuracy(name="accuracy"))

log_dir = "logs/fit/VGG" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
callback = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True) 

model.fit(train_ds,
            epochs=epochs,
            verbose=1,
            validation_data=val_ds,
            callbacks=[tensorboard_callback, callback])

## Hyperparameter Tuning

We can also utilize the tensorboard display to give us a view of hyperparameter tuning. This requires more work than a simple grid search, but the results are pretty similar. Below is an example adapted from the tensorflow docs. 

We'll do this with a simple model - a dense layer, a dropout, and the output, more complex ones are the same in their setup:
<ol>
<li> HP_NUM_UNITS - test a different number of units between 16 and 64. 
<li> HP_DROPOUT - the proportion of dropouts in the dropout. 
<li> HP_OPTIMIZER - we can try some different optimizers. 
</ol>

In [ ]:
# Load some data
from tensorboard.plugins.hparams import api as hp
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#### Setup Parameters

We can define the parameters we want to grid search here. Each one is one of these hParam objects - we assign it a name and a range of values to use, here we have a numerical and a discreet example. We list those variables in the hparams argument. 

This is very similar to the idea of setting different values in a gridsearch, just with slightly different syntax. 

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32, 48, 64]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.4))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', "rmsprop"]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

#### Build Test Models

We can create our models inside of some helper functions - each one will run a model with certain HPs and return the accuracy, or whichever other metric we define. 

Note the key change - the variables that we are changing are replaced with the matching hparams item. 

In [ ]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, epochs=10) 
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

#### Perfrom the GridSearch

We have to write the gridsearch manually, but we can copy this basic setup as a template and modify it. Once complete, load tensorboard and go to the HPARAMS section to visualize. 

The parallel coordinates view allows us to do a quick exploration of the best HPs. 

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1

## Keras Tuner

In addition to the fairly manual version of a gridseach we defined above, we can also use a library called KerasTuner to do a similar tuning process, but with a bit more automation.